# Твиты

1) В качестве брокера сообщений использовать Kafka.

2) Для работы с потоком твитов - пакет tweepy под Python.

## Подготовка

## Задача 5. Подсчет количества ретвитов на твиты пользователей

Напишите программу, которая подсчитывает количество ретвитов на твиты каждого пользователя в течение 1 мин. с накоплением.

Выведите результат в отсортированном по убыванию виде: id твита, количество ретвитов. Каждую 1 мин. сохранять результат в файл. Накапливать не менее 15 мин.

После этого для 5 наиболее популярных твитов вывести: screen_name пользователя, id твита и его текст.

Исходные данные:
- id пользователей: "285532415", "147964447", "34200559", "338960856", "200036850", "72525490", "20510157", "99918629"

Запустим **tweets_producer.py**, созданный для задания 4.

В "счетчике ретвитов" используется *tweepy* для получения актуальной информации о твите.

**retweets_counter.py**

```python
import sys
import time

from datetime import datetime

import rapidjson
import tweepy

from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

from local_settings import *

ZOOKEEPER_SERVER = 'localhost:2181'
RETWEETS_KAFKA_TOPIC = 'Retweets'

APP_NAME = 'UsersRetweetsCounter'
MINUTE = 60
BATCH_DURATION_SEC = MINUTE

# Init twitter API
auth = tweepy.OAuthHandler(CONSUMER_TOKEN, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

api = tweepy.API(auth)

# Init streaming
spark_context = SparkContext(appName=APP_NAME)
spark_context.setLogLevel('ERROR')

streaming_context = StreamingContext(spark_context, BATCH_DURATION_SEC)
streaming_context.checkpoint(f'{APP_NAME}__checkpoint')

# Process tweets
kafka_stream = KafkaUtils.createStream(
	streaming_context,
	ZOOKEEPER_SERVER,
	f'{APP_NAME}__consumers_group',
	{RETWEETS_KAFKA_TOPIC: 1},
)

# [
#	(None, '1074310692235292672'),
#	(None, '1074310709134139392')
#	(None, '1074310692235292672')
# ] => [
#	('1074310692235292672', 2)
# 	('1074310709134139392', 1)
# ]
tid__retweets_count = kafka_stream.map(lambda none__tid: (none__tid[1], 1)).reduceByKey(lambda a, b: a + b)


def update_retweets_count(new_retweets_counts, retweets_count__in_state):
	retweets_count__in_state = retweets_count__in_state or 0
	return sum(new_retweets_counts, retweets_count__in_state)


tid__retweets_total_count = tid__retweets_count.updateStateByKey(update_retweets_count)


def sort_by_retweets_count(tid__retweets_count):
	return tid__retweets_count[1]


top_tweets_by_retweets = tid__retweets_total_count.transform(
    lambda rdd: rdd.sortBy(sort_by_retweets_count, ascending=False)
)
# Every minute print 10 of top retweeted tweets and save result in file
top_tweets_by_retweets.pprint(10)
top_tweets_by_retweets.transform(lambda rdd: rdd.coalesce(1)).saveAsTextFiles('file:///tmp/retweets/counts')

# Every 15 minutes print info about 5 of top retweeted tweets
windowed_data = top_tweets_by_retweets.reduceByKeyAndWindow(
	max,
	None,
	windowDuration=MINUTE * 15,
	slideDuration=MINUTE * 15
)


def get_and_print_top_tweets_info(rdd, limit=5):
	for tid__retweets_count in rdd.sortBy(sort_by_retweets_count, ascending=False).take(limit):
		tid = tid__retweets_count[0]
		try:
			tweet = api.get_status(tid)
		except tweepy.error.TweepError:
			print(f'Tweet #{tid} was deleted')
			return

		text = tweet.text
		extended_tweet = getattr(tweet, 'extended_tweet', None)
		if extended_tweet:
			text = extended_tweet.full_text

		print(f'\n@{tweet.user.screen_name} tweeted in #{tid}:\n"""\n{tweet.text}\n"""\n')


windowed_data.transform(lambda rdd: rdd.coalesce(1)).foreachRDD(
    lambda _, rdd: get_and_print_top_tweets_info(rdd)
)

# Start and deinit later
streaming_context.start()
streaming_context.awaitTermination()
```

Отобразим файлы с промежуточными ежеминутными результатами.

```bash
[cloudera@quickstart src]$ ls -l /tmp/retweets/
total 68
drwxrwxr-x 2 cloudera cloudera 4096 Dec 19 08:37 counts-1545237420000
drwxrwxr-x 2 cloudera cloudera 4096 Dec 19 08:38 counts-1545237480000
drwxrwxr-x 2 cloudera cloudera 4096 Dec 19 08:39 counts-1545237540000
drwxrwxr-x 2 cloudera cloudera 4096 Dec 19 08:40 counts-1545237600000
drwxrwxr-x 2 cloudera cloudera 4096 Dec 19 08:41 counts-1545237660000
drwxrwxr-x 2 cloudera cloudera 4096 Dec 19 08:42 counts-1545237720000
drwxrwxr-x 2 cloudera cloudera 4096 Dec 19 08:43 counts-1545237780000
drwxrwxr-x 2 cloudera cloudera 4096 Dec 19 08:44 counts-1545237840000
drwxrwxr-x 2 cloudera cloudera 4096 Dec 19 08:45 counts-1545237900000
drwxrwxr-x 2 cloudera cloudera 4096 Dec 19 08:46 counts-1545237960000
drwxrwxr-x 2 cloudera cloudera 4096 Dec 19 08:47 counts-1545238020000
drwxrwxr-x 2 cloudera cloudera 4096 Dec 19 08:48 counts-1545238080000
drwxrwxr-x 2 cloudera cloudera 4096 Dec 19 08:49 counts-1545238140000
drwxrwxr-x 2 cloudera cloudera 4096 Dec 19 08:50 counts-1545238200000
drwxrwxr-x 2 cloudera cloudera 4096 Dec 19 08:51 counts-1545238260000
drwxrwxr-x 2 cloudera cloudera 4096 Dec 19 08:52 counts-1545238320000
drwxrwxr-x 2 cloudera cloudera 4096 Dec 19 08:53 counts-1545238380000
```

Выведем результаты файла, соответствующего 7 итерации:

```text
-------------------------------------------
Time: 2018-12-19 08:43:00
-------------------------------------------
('1075428203794194434', 9)
('1075419176922169349', 6)
('1075420936629821440', 5)
('1075429278316150784', 5)
('1075423970948722696', 4)
('1075430675401097216', 4)
('1075429670970105856', 3)
('1075429801865945088', 3)
('1075430068502085635', 3)
('1075429016310558721', 2)
...
```

```bash
[cloudera@quickstart src]$ cat /tmp/retweets/counts-1545237780000/part-00000 | head -10
('1075428203794194434', 9)
('1075419176922169349', 6)
('1075420936629821440', 5)
('1075429278316150784', 5)
('1075423970948722696', 4)
('1075430675401097216', 4)
('1075429670970105856', 3)
('1075429801865945088', 3)
('1075430068502085635', 3)
('1075429016310558721', 2)
```

Таким образом, программа верно выводит количество ретвитов для твитов **за последние 15 минут**,

при этом обновление статистики выполняется **каждую минуту** (при этом она выгружается в файл),

а общее количество ретвитов для каждого твита **накапливается между итерациями**.

В конце верно выводится 5 наиболее популярных твитов с информацией об их авторе.
